
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [1]:

# Part 1

# Load the data from the file specified in the problem definition and make sure that it is loaded using
# the search path `data/Corona_NLP_train.csv`. This is to make sure the autograder and your computer have the same
# file path and can load the data correctly.

# Contrary to how many other problems are structured, this problem actually requires you to
# have X on the shape (n_samples, ) that is a 1-dimensional array. Otherwise it will cause a bunch
# of errors in the autograder or also in for instance CountVectorizer.

# Make sure that all your data is numpy arrays and not pandas dataframes or series.

import pandas as pd 
df = pd.read_csv('data/Corona_NLP_train.csv', encoding='MacRoman')
df = df[df['Sentiment'] != 'Neutral']
X = df['OriginalTweet']
Y = df['Sentiment']
Y = Y.map({'Positive': 1, 'Extremely Negative': 0, 'Negative': 0, 'Extremely Positive': 1})
X = X.to_numpy()
Y = Y.to_numpy()

train_end = int(0.6 * len(X))
test_end = train_end + int(0.15 * len(X))

X_train = X[:train_end]
Y_train = Y[:train_end]
X_test = X[train_end:test_end]
Y_test = Y[train_end:test_end]
X_valid = X[test_end:]
Y_valid = Y[test_end:]

print(X)

['advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order'
 'Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P'
 "My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\r\r\n#COVID19france #COVID_19 #COVID19 #coronavirus #confinement #Confinementotal #ConfinementGeneral https://t.co/zrlG0Z520j"
 ...
 'Response to complaint not provided citing COVID-19 related delays. Yet prompt in rejecting policy before consumer TAT is over. Way to go ?'
 'You know it¬ís getting tough when @KameronWilds  is rationing toilet paper #coronavirus #toiletpaper @kroger martinsville, help us out!!'
 "@TartiiCat

In [2]:

# Part 2

# Train a machine learning model or pipeline that can take the raw strings from X and predict Y=0,1 depending on the
# sentiment of the tweet. Store the trained model in the variable `model`.
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

pipe = Pipeline(
    [
        ('vectorizer', CountVectorizer(encoding='MacRoman')),
        ('logreg', LogisticRegression())
    ]
)

params = {
    'logreg__C': [0.01, 0.1, 1, 10],
    'logreg__penalty': ['l2', 'l1'],
    'logreg__solver': ['liblinear'],
    'logreg__class_weight': [
        {0:1, 1:1}, {0:1, 1:2}, {0:1, 1:2.5}, {0:1, 1:3}, {0:1, 1:4}, 'balanced'
    ],
}

grid = GridSearchCV(
    pipe,
    params,
    cv=5,
    scoring='f1',
    n_jobs=-1
)

grid.fit(X_train, Y_train)
print(grid.param_grid)

model = grid.best_estimator_


{'logreg__C': [0.01, 0.1, 1, 10], 'logreg__penalty': ['l2', 'l1'], 'logreg__solver': ['liblinear'], 'logreg__class_weight': [{0: 1, 1: 1}, {0: 1, 1: 2}, {0: 1, 1: 2.5}, {0: 1, 1: 3}, {0: 1, 1: 4}, 'balanced']}


In [3]:

# Part 3

# Evaluate the model on the test set and calculate precision, and recall on both classes. Store the results in the
# variables `precision_0`, `precision_1`, `recall_0`, `recall_1`.
from sklearn.metrics import precision_recall_fscore_support
Y_pred = model.predict(X_test)
precision, recall, f1, support = precision_recall_fscore_support(
    Y_test, Y_pred, labels=[0, 1]
)
precision_0 = precision[0]
precision_1 = precision[1]
recall_0 = recall[0]
recall_1 = recall[1]

In [4]:

# Part 4
# c(1 - Precision)\prob{g(X)=1} + d(1 - Recall)\prob{Y=1}.
def cost(model,threshold,X,Y):
    # Hint, make sure that the model has a predict_proba method
    # think about how the decision is made based on the probabilities
    # and how the threshold can be used to make the decision.
    # For reference take a look at the lecture notes "Bayes classifier"
    # which contains how the decision is made based on the probabilities when the threshold is 0.5.
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost

    P_Y_given_X = model.predict_proba(X)[:, 1]
    Y_pred = (P_Y_given_X > threshold).astype(int)
    FN = (Y == 1) & (Y_pred == 0)
    FP = (Y == 0) & (Y_pred == 1)
    
    return (FN.sum() + FP.sum() * 5) / len(Y)

In [5]:
import numpy as np
# Part 5

# Find the optimal threshold for the model on the test set. Store the threshold in the variable `optimal_threshold`
# and the cost at the optimal threshold in the variable `cost_at_optimal_threshold` evaluated on the test set.

thresholds = np.linspace(0, 1, 1001)
costs = []
for t in thresholds:
    c = cost(model, t, X_test, Y_test)
    costs.append(c)

costs = np.array(costs)
optimal_threshold = thresholds[np.argmin(costs)]
cost_at_optimal_threshold = costs.min()

In [6]:

# Part 6

cost_at_optimal_threshold_valid = cost(model, optimal_threshold, X_valid, Y_valid)
delta = 0.01
n = len(Y_valid)
cost_range = 5
epsilon = cost_range * np.sqrt(np.log(2 / delta) / (2 * n))
cost_interval_valid = (cost_at_optimal_threshold_valid - epsilon, cost_at_optimal_threshold_valid + epsilon)

assert(type(cost_interval_valid) == tuple)
assert(len(cost_interval_valid) == 2)

In [7]:

# Part 7
probs = model.predict_proba(X_valid)[:,1]
C = (1 - (probs >= optimal_threshold).astype(int)) * Y_valid + 5 * (1 - Y_valid) * (probs >= optimal_threshold).astype(int)

variance_of_C = np.var(C, ddof=1)

n = len(C)
b = 5

delta = 0.01
epsilon = np.sqrt(2 * variance_of_C * np.log(2/delta) / n) + b * np.log(2/delta) / (3*n)
interval_of_C = (C.mean() - epsilon, C.mean() + epsilon)

assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)

In [8]:
# ASSIGNMENT 4, TEST 1, POINTS 24

Numerical error -2.6020852139652106e-18
Numerical error -2.6020852139652106e-18
Beginning tests for problem 1

---------------------------------
Beginning test for part1
---------------------------------

-----Beginning test------
X.shape is correct, it is (33444,)
-----Ending test---------

-----Beginning test------
Y.shape is correct, it is (33444,)
-----Ending test---------

-----Beginning test------
X seems to be a list of strings
-----Ending test---------

-----Beginning test------
Y contains the correct values {0,1}
-----Ending test---------

-----Beginning test------
Shape of X_train correct it is (20066,)
-----Ending test---------

-----Beginning test------
Shape of X_test correct it is (5016,)
-----Ending test---------

-----Beginning test------
Shape of X_valid correct it is (8362,)
-----Ending test---------

-----Beginning test------
Shape of Y_train correct it is (20066,)
-----Ending test---------

-----Beginning test------
Shape of Y_test correct it is (5016,)
-----Ending 

The number of points you have scored in total for this entire set of Problems is 24 out of 24.